In [19]:
import requests
import pandas as pd
import numpy as np
import math
import json
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
from IPython.display import Markdown as md
import os

output_notebook()
url = "http://bing.com/covid/data"
r = requests.get(url)
j = r.json()
with open('state_populations.json') as data:
    state_pop = pd.read_json(data)
#j = r.json()
#print(j)
#df = pd.DataFrame(j['areas'][2])
df = pd.json_normalize(j['areas'][0], 'areas')
df.fillna(0, inplace=True)
#df['totalConfirmed'].sum()
index = df['displayName'].to_list()
totalConfirmed = df['totalConfirmed'].to_list()
totalDeaths = df['totalDeaths'].to_list()
totalRecovered = df['totalRecovered'].to_list()
df2 = pd.DataFrame({'totalConfirmed': totalConfirmed,
                    'totalDeaths': totalDeaths,
                    'totalRecovered': totalRecovered}, index=index)
df2['totalRecovered'].fillna(0, inplace=True)
df2['deathRate'] = df2['totalDeaths'].div(df2['totalConfirmed'], fill_value=0)
df2['recoveryRate'] = df2['totalRecovered'].div(df2['totalConfirmed'], fill_value=0)
covid = pd.merge(left=df2, right=state_pop[['state_name','population']], how='left', left_index=True, right_on='state_name')
covid['infectionRate'] = covid['totalConfirmed'].div(covid['population'], fill_value=0)
covid = covid[['state_name', 'population', 'totalConfirmed', 'infectionRate', 'totalDeaths', 'deathRate', 'totalRecovered', 'recoveryRate']]
covid.set_index('state_name', inplace=True)

total_population = int(covid['population'].sum())
total_confirmed = int(covid['totalConfirmed'].sum())
total_deaths = int(covid['totalDeaths'].sum())
total_recoveries = int(covid['totalRecovered'].sum())
infection_rate = total_confirmed / total_population
death_rate = total_deaths / total_confirmed
recovery_rate = total_recoveries / total_confirmed

covid['expectedCases'] = covid['population'].mul(infection_rate)
covid['caseDelta'] = covid['totalConfirmed'].sub(covid['expectedCases'])
covid['caseDeltaRate'] = covid['caseDelta'].div(covid['expectedCases'], fill_value=0)

covid['expectedDeaths'] = covid['totalConfirmed'].mul(death_rate)
covid['deathDelta'] = covid['totalDeaths'].sub(covid['expectedDeaths'])
covid['deathDeltaRate'] = covid['deathDelta'].div(covid['expectedDeaths'], fill_value=0)

covid['caseShare'] = covid['totalConfirmed'].div(total_confirmed, fill_value=0)
covid['deathShare'] = covid['totalDeaths'].div(total_deaths, fill_value=0)
covid['recoveryShare'] = covid['totalRecovered'].div(total_recoveries, fill_value=0)

Loading BokehJS ...

In [23]:
states_to_watch = ["Tennessee", "Kentucky", "Indiana"]
stat_tuples_array = [('Infection Rate', "infectionRate", infection_rate, 0), 
                     ('Death Rate', "deathRate", death_rate, 0),
                     ('Case Delta', "caseDelta", '-----', 1),
                     ('Delta %', "caseDeltaRate", '-----', 0),
                     ('Expected Cases', "expectedCases", '-----', 1),
                     ('Actual Cases', "totalConfirmed", total_confirmed, 1),
                     ('Deaths Delta', "deathDelta", '-----', 1),
                     ('Delta %', "deathDeltaRate", '-----', 0),
                     ('Expected Deaths', "expectedDeaths", '-----', 1),
                     ('Actual Deaths', "totalDeaths", total_deaths, 1),
                     ('Case Share', "caseShare", '-----', 0),
                     ('Death Share', "deathShare", '-----', 0),
                     ('Recovery Share', "recoveryShare", '-----', 0),
                    ]
table = " | Metric | "
for state in states_to_watch:
    table += "`{}` | ".format(state)
table += "`United States` |\n"
table += "| --- | "
for state in states_to_watch:
    table += "---: | "
table += "---: |\n"
for field in stat_tuples_array:
    table += "| {} | ".format(field[0])
    if field[3] == 0:
        for state in states_to_watch:
            table += "`{:.4%}` | ".format(covid.loc[state, format(field[1])])
        if isinstance(field[2], float):
            table += "`{:.4%}` |\n".format(field[2])
        else:
            table += "`{}` |\n".format(field[2])
    elif field[3] == 1:
        for state in states_to_watch:
            table += "`{:,}` | ".format(int(covid.loc[state, field[1]]))
        if isinstance(field[2], float) or isinstance(field[2], int):
            table += "`{:,}` |\n".format(int(field[2]))
        else:
            table += "`{}` |\n".format(field[2])

md(table)

 | Metric | `Tennessee` | `Kentucky` | `Indiana` | `United States` |
| --- | ---: | ---: | ---: | ---: |
| Infection Rate | `0.0139%` | `0.0055%` | `0.0096%` | `0.0247%` |
| Death Rate | `0.3135%` | `0.0000%` | `2.6357%` | `1.3957%` |
| Case Delta | `-743` | `-861` | `-1,017` | `-----` |
| Delta % | `-43.7210%` | `-77.6437%` | `-61.2130%` | `-----` |
| Expected Cases | `1,700` | `1,109` | `1,662` | `-----` |
| Actual Cases | `957` | `248` | `645` | `81,680` |
| Deaths Delta | `-10` | `-3` | `7` | `-----` |
| Delta % | `-77.5395%` | `-100.0000%` | `88.8426%` | `-----` |
| Expected Deaths | `13` | `3` | `9` | `-----` |
| Actual Deaths | `3` | `0` | `17` | `1,140` |
| Case Share | `1.1716%` | `0.3036%` | `0.7897%` | `-----` |
| Death Share | `0.2632%` | `0.0000%` | `1.4912%` | `-----` |
| Recovery Share | `0.0000%` | `0.0000%` | `0.0000%` | `-----` |


In [7]:
source = ColumnDataSource(data=covid)
plot_height = 600
plot_width = 1000
width = 0.5

title_p = "COVID Confirmed: {:,} - Death Rate: {:.4%} - Recovery Rate: {:.4%}".format(total_confirmed, death_rate, recovery_rate)
title_q = "COVID Deaths: {:,} - Death Rate: {:.4%}".format(total_deaths, death_rate)
title_z = "COVID Recovered: {:,} - Recovery Rate: {:.4%}".format(total_recoveries, recovery_rate)
title_x = "COVID US National Infection Rate: {:.4%}".format(infection_rate)
title_y = "COVID US Confirmed / Expected Delta"
title_m = "COVID US Percentage Share of Confirmed Cases by State"
title_n = "COVID US Percentage Share of Deaths by State"
title_o = "COVID US Percentage Share of Recoveries by State"

p = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_p)
p.vbar(x='state_name', top='totalConfirmed', width=width, source=source, line_color='white')
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = covid['totalConfirmed'].max() + covid['totalConfirmed'].std()
p.xaxis.major_label_orientation = math.pi / 4

q = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_q)
q.vbar(x='state_name', top='totalDeaths', width=width, source=source, line_color='white')
q.xgrid.grid_line_color = None
q.y_range.start = 0
q.y_range.end = covid['totalDeaths'].max() + covid['totalDeaths'].std()
q.xaxis.major_label_orientation = math.pi / 4

z = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_z)
z.vbar(x='state_name', top='totalRecovered', width=width, source=source, line_color='white')
z.xgrid.grid_line_color = None
z.y_range.start = 0
z.y_range.end = covid['totalRecovered'].max() + covid['totalRecovered'].std()
z.xaxis.major_label_orientation = math.pi / 4

x = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_x)
x.vbar(x='state_name', top='infectionRate', width=width, source=source)
x.xgrid.grid_line_color = None
x.y_range.start = 0
x.y_range.end = covid['infectionRate'].max() + covid['infectionRate'].std()
x.xaxis.major_label_orientation = math.pi / 4

y = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_y)
y.vbar(x='state_name', top='caseDelta', width=width, source=source)
y.xgrid.grid_line_color = None
y.y_range.start = covid['caseDelta'].min() - covid['caseDelta'].std()
y.y_range.end = covid['caseDelta'].max() + covid['caseDelta'].std()
y.xaxis.major_label_orientation = math.pi / 4

m = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_m)
m.vbar(x='state_name', top='caseShare', width=width, source=source)
m.xgrid.grid_line_color = None
m.y_range.start = 0
m.y_range.end = covid['caseShare'].max() + covid['caseShare'].std()
m.xaxis.major_label_orientation = math.pi / 4

n = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_n)
n.vbar(x='state_name', top='deathShare', width=width, source=source)
n.xgrid.grid_line_color = None
n.y_range.start = 0
n.y_range.end = covid['deathShare'].max() + covid['deathShare'].std()
n.xaxis.major_label_orientation = math.pi / 4

o = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_o)
o.vbar(x='state_name', top='recoveryShare', width=width, source=source)
o.xgrid.grid_line_color = None
o.y_range.start = 0
o.y_range.end = covid['recoveryShare'].max() + covid['recoveryShare'].std()
o.xaxis.major_label_orientation = math.pi / 4

show(x)
show(y)
show(m)
show(n)
show(p)
show(q)
show(z)
show(o)